In [ ]:
# Install dependencies
import os, time
start_t=time.time()
start_to=time.time()
os.system('pip install -q pydca')
os.system('pip install -q pyBigWig')
os.system('pip install -q numpy -U')

In [ ]:
#Download the compartment information from PyMEGABASE github
!git clone https://github.com/ed29rice/PyMEGABASE.git

In [ ]:
print('Done installing and retrieving PyMEGABASE')
print('Enlapsed time:',time.time()-start_t)
start_t=time.time()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PyMEGABASE.PyMEGABASE as PYM
import PyMEGABASE.drop_down as dp
import urllib.parse

In [ ]:
#First let's look for the desired cell line in the ENCODE database
cell_lines_menu=dp.cell_lines(assembly='hg19', signal_type='signal p-value',
                            histones=True,tf=False,small_rna=True,total_rna=True)


In [ ]:
print('Please select one cell line from the drop down menu')
cell_lines_menu.menu

In [ ]:
#Run this cell after chosing the the cell line above
cell_line=cell_lines_menu.menu.value
print('You selected the following cell line:')
print(cell_line)
#Translate name of cell line to url version of the name
cell_line_url_name=urllib.parse.quote_plus(cell_line)

In [ ]:
#Initialize PyMEGABASE 
pym=PYM.PyMEGABASE_extended(cell_line=cell_line_url_name, assembly='hg19', signal_type='signal p-value',
                            ref_cell_line_path='training',types_path='PyMEGABASE/types',
                            histones=True,tf=False,atac=False,small_rna=True,total_rna=True) 

In [ ]:
#Download data for the selected cell line from ENCODE
pym.download_and_process_cell_line_data(nproc=5)

In [ ]:
#Download data for the reference cell line (GM12878) from ENCODE
pym.download_and_process_ref_data(nproc=5)

In [ ]:
print('Done downloading data')
print('Enlapsed time:',time.time()-start_t)
start_t=time.time()

In [ ]:
#Preprocess the downloaded data for tranining
pym.training_set_up()

In [ ]:
#Perform the training using the downloaded reference data
pym.training(nproc=5)

In [ ]:
print('Done training')
print('Enlapsed time:',time.time()-start_t)
start_t=time.time()

In [ ]:
#Choose the chromosome you want to get the prediction
chr=2
#Predict
types_pyME=pym.prediction(chr=2)

#Show the prediction results
x=[i for i in range(1,len(types_pyME)+1)]
plt.figure(figsize=(20,5))
plt.scatter(np.array(x)*50000/10**6,types_pyME,s=5,c=types_pyME, cmap=plt.get_cmap('Set1'))
plt.yticks(np.array([0,1,2,3,4]),['A1','A2','B1','B2','B3'])
plt.ylim([-0.4,4.2])

plt.figure(figsize=(20,5))
plt.scatter(np.array(x)*50000/10**6,types_pyME,s=100,marker='s',c=types_pyME, cmap=plt.get_cmap('Set1'))
plt.yticks(np.array([0,1,2,3,4]),['A1','A2','B1','B2','B3'])
plt.ylim([-0.4,4.2])
plt.xlim([71.7,73.2])

fig, axs = plt.subplots(1, 2,figsize=(20,8))
type_list, counts = np.unique(types_pyME,return_counts=True)
axs[0].bar(type_list,counts/len(types_pyME))
preAB=np.copy(types_pyME)
preAB[np.where((types_pyME==0) | (types_pyME==1))]=1
preAB[np.where((types_pyME==2) | (types_pyME==3) | (types_pyME==4))]=0
type_list, counts = np.unique(preAB,return_counts=True)
axs[1].bar(['B','A','NA'],counts/len(preAB))

In [ ]:
#Predict and save the data 
TYPE_TO_INT = {'A1':0,'A2':1,'B1':2,'B2':3,'B3':4,'B4':5,'NA':6}
INT_TO_TYPE = {TYPE_TO_INT[k]:k for k in TYPE_TO_INT.keys()}

#Go over chromosomes
for chr in range(1,23):
    types_pyME=pym.prediction(chr)
    types_pyME_letters=np.array(list(map(INT_TO_TYPE.get, types_pyME)))
    #Save data
    with open('chr'+str(chr)+'_beads.txt','w',encoding = 'utf-8') as f:
        for i in range(len(types_pyME)):
            f.write("{} {}\n".format(i+1,types_pyME_letters[i]))

types_pyME=pym.prediction_X()
types_pyME_letters=np.array(list(map(INT_TO_TYPE.get, types_pyME)))
#Save data
with open('chr'+str(chr)+'_beads.txt','w',encoding = 'utf-8') as f:
    for i in range(len(types_pyME)):
        f.write("{} {}\n".format(i+1,types_pyME_letters[i]))


print('Done predicting')
print('Enlapsed time:',time.time()-start_t)


# If you want to save your results on Google Drive run the next cell

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

drive_folder_name='PyMEGABASE_'+pym.cell_line+'_'+pym.assembly
gfolder = drive.CreateFile({'title':drive_folder_name,'mimeType': 'application/vnd.google-apps.folder'})
gfolder.Upload()

upload_file_list=['chr'+str(chr)+'_beads.txt' for chr in range(1,23)]
for upload_file in upload_file_list:
	gfile = drive.CreateFile({'parents': [{'id': gfolder.get('id')}]})
	# Read file and set it as the content of this instance.
	gfile.SetContentFile(upload_file)
	gfile.Upload() # Upload the file.

In [ ]:
print('Done with everything')
print('Enlapsed time:',time.time()-start_to)

# Run only if predicting GM12878 - hg19 

In [ ]:
#Define tranlation dictinaries between aminoacids, intensity of Chip-seq signal and 
TYPE_TO_INT = {'A1':0,'A2':1,'B1':2,'B2':3,'B3':4,'B4':5,'NA':6}

INT_TO_TYPE = {TYPE_TO_INT[k]:k for k in TYPE_TO_INT.keys()}
count_M=0;count_P=0;c=0;cc=0

for chr in range(2,23,2):
    print('\nPredicting chromosome:',chr)
    types_pyME=pym.prediction(chr)
    types_original=np.loadtxt('./PyMEGABASE/types/chr'+str(chr)+'_beads.txt.original',delimiter=' ',dtype=str)[:,1]
    #Translate them into states
    int_types_Or=np.array(list(map(TYPE_TO_INT.get, types_original)))
    idx=(int_types_Or!=6)
    print('Accuracy of PyMEGABASE Rao et al 2014:',np.round(np.sum(types_pyME[idx]==int_types_Or[idx])/len(int_types_Or[idx]),4))
    count_P=count_P+np.sum(types_pyME[idx]==int_types_Or[idx])
    c=c+np.sum(idx)
  
print('=====================================================')
print('Accuracy of PyMEGABASE across chromosomes:',np.round(count_P/c,4))
print('=====================================================')

In [ ]:
chr=2
types_pyME=pym.prediction(chr)
types_original=np.loadtxt('./PyMEGABASE/types/chr'+str(chr)+'_beads.txt.original',delimiter=' ',dtype=str)[:,1]
types_ME=np.loadtxt('./PyMEGABASE/GM12878_Original_MEGABASE/chr'+str(chr)+'_beads.txt',delimiter=' ',dtype=str)[:,1]
int_types_Or=np.array(list(map(TYPE_TO_INT.get, types_original)))
int_types_ME=np.array(list(map(TYPE_TO_INT.get, types_ME)))

idx=(int_types_Or!=6)
plt.figure(figsize=(25,7))
plt.plot(int_types_Or[idx],'s',markersize=1,label='Rao et al 2014')
plt.plot(int_types_ME[idx]-0.1,'s',markersize=1, label='MEGABASE')
plt.plot(types_pyME[idx]-0.2,'s',markersize=1, label='PyMEGABASE')
plt.yticks(np.array([0,1,2,3,4])-0.1,['A1','A2','B1','B2','B3'])
plt.ylim([-0.4,4.2])
plt.xlim([0,len(int_types_Or[idx])])
plt.legend(loc='upper right')